# Preprocessing for SQLite Database

In this notebook, we read in the raw script and subtitles files and process the files in order to get the basic information from each file. Then we input the clean, organized data into a SQLite database. 

In [ ]:
from bs4 import BeautifulSoup
import IPython
import re
import sqlite3
import os

### Creating friendsdb SQLite database and script table

In [ ]:
conn = sqlite3.connect('/Users/Jack/Developer/friends/friendsdb.sqlite')
cur = conn.cursor()

cur.executescript('''
CREATE TABLE IF NOT EXISTS script (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    season INTEGER,
    episode INTEGER,
    linecount INTEGER,
    char TEXT,
    words TEXT,
    UNIQUE(season, episode, linecount)
);
''')

### Utility for finding multiple characters in a paragraph in script

In [ ]:
def find_nth(string, substring, n):
   if (n == 1):
       return string.find(substring)
   else:
       return string.find(substring, find_nth(string, substring, n - 1) + 1)

### Preprocessing Script File

We read in the scripts one by one and use Beautiful Soup to clean up the HTML. Then we apply regex to separate the characters from the spoken lines. There are a ton of inconsistencies and irregularities in the script files so the regex expressions and various other filters are quite numerous. But eventually we output a dictionary where the keys delineate the specific episode and the values are lists containing the character and the line spoken by that character. 

In [ ]:
count = 0
scriptsfolder = '/Users/Jack/Developer/friends/scripts/'
masterscriptsdict = {}
listofallchars = []
episodelist = []

for filename in os.listdir(scriptsfolder):
    if not filename.startswith('.'):
        season = filename[:2]
        episode = filename[2:4]

        preSQLscript = []
        rawlines = []
#         print("season is " + str(season))
#         print("episode is " + str(episode))
    #     count += 1
    #     print("count is " + str(count))

        htmlpath = scriptsfolder + filename
        # Note: we get errors unless we specify latin-1 encoding
        with open(htmlpath, "r", encoding='latin-1') as s:
            contents = s.read()
            soup = BeautifulSoup(contents, 'lxml')

            for body in soup.find_all("body"):
                text = body.text
                lines = text.split("\n")
#                     if season == "09" and episode == "08":
#                         print(lines)
                pattern = re.compile(r"^[A-Z][a-zA-Z,. ]+:")
                for line in lines:
                    # remove everything in () from line
                    line = re.sub("\([^)]*\)", "", line)
                    # remove everything in [] from line
                    line = re.sub("\[[^]]*\]", "", line)
                    # remove everything in <> from line
                    line = re.sub("\<[^]]*\>", "", line)
                    # remove everything in {} from line
                    line = re.sub("\{[^]]*\}", "", line)
                    # remove everything in [) from line, yes some transcriber used this nomenclature
                    line = re.sub("\[[^]]*\)", "", line)
                    line = line.strip()
                    # check for " by:"
                    if " by:" in line.lower():
                        line = ""
                    # split char from line
                    if re.search(pattern, line):
                        rawlines.append([line])
                    else:
                        if rawlines and line:
                            if line.lower().strip() not in ["opening credits", "opening titles", "closing credits", "closing titles", "commercial break", "end"]:
                                rawlines[-1].append(line)
                rawlines = [" ".join(linegroup) for linegroup in rawlines]
                rawlines = [re.sub(r'[^\x00-\x7f]',r' ', line)  for line in rawlines]
                for line in rawlines:
                    # remove everything in () from line
                    line = re.sub("\([^)]*\)", "", line)
                    # remove everything in [] from line
                    line = re.sub("\[[^]]*\]", "", line)
                    # remove everything in <> from line
                    line = re.sub("\<[^]]*\>", "", line)
                    # remove everything in {} from line
                    line = re.sub("\{[^]]*\}", "", line)
                    # remove everything in [) from line, yes some transcriber used this nomenclature
                    line = re.sub("\[[^]]*\)", "", line)
                    # split char from line
                    line = line.strip()
                    if ":" in line:
                        colonsplit = line.split(':', 1)
                        char = colonsplit[0]
                        line = colonsplit[1]
    #                     make all words lowercase
                        line = line.lower()
                        line = line.strip()
                        # remove all punctuation
                        line = re.sub("[^\w]", " ",  line)

                        # Dealing with Season 9, Episode 8 super long char names
                        if season == "09" and episode == "08":
#                                 print(char)
                            charlist = char.split()
                            if len(charlist) > 2:
                                if charlist[1] == "and":
                                    char = ' '.join(charlist[0:3])
                                else:
                                    char = charlist[0]
                            else:
                                char = charlist[0]

                        # To easily see all char names (for checking)
                        if char not in listofallchars:
                            listofallchars.append(char)

                        if [season + episode] not in episodelist:
                            episodelist.append([season + episode])

                        # adding char and their line to the database
                        preSQLscript.append([char, line])

                if (season == "09" and episode == "08"):
                    print(preSQLscript)

        masterscriptsdict[season + episode] = preSQLscript
# print(len(episodelist))
# print(sorted(episodelist))

### Putting each line of script into script table in friendsdb database

In [ ]:
count = 0
for key, script in masterscriptsdict.items():
    season = int(key[0:2])
    episode = int(key[2:4])
    count += 1
    print(count)
    for i, line in enumerate(script):
        linecount = i
        char = line[0]
        words = line[1]
        cur.execute('''INSERT OR REPLACE INTO script (season, episode, linecount, char, words)
            VALUES ( ?, ?, ?, ?, ? )''', ( season, episode, linecount, char, words ) )
        conn.commit()

### Now repeating the process for the subtitles files

We begin by creating the subtitles table in the SQLite database. This table will eventually contain our "guesses" for which character speaks a specific line.

In [ ]:
# Creating subs SQLite table
conn = sqlite3.connect('/Users/Jack/Developer/friends/friendsdb.sqlite')
cur = conn.cursor()

cur.executescript('''
CREATE TABLE IF NOT EXISTS subs (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    season INTEGER,
    episode INTEGER,
    starttime TEXT,
    endtime TEXT,
    x TEXT,
    y TEXT,
    linenum INTEGER,
    line TEXT,
    UNIQUE(season, episode, linenum)
);
''')

### Defining a few functions that will help us process the subtitles file

In [ ]:
def getcharfromline(line):
    char = ""
    lineparthascolon = line.find(":")
    if lineparthascolon != -1:
        if line[lineparthascolon - 1].isupper():
            char = line[:lineparthascolon]
            wordsstart = lineparthascolon + 1
            line = line[wordsstart:].strip()
    return char, line

def startswithdash(line):
    if line:
        if line[0] == "-":
            return True
        else:
            return False

def preprocess(line):
    # make lowercase
    line = line.lower()
    # remove everything in () from words
    line = re.sub("\([^)]*\)", "", line)
    # remove everything in [] from words
    line = re.sub("\[[^]]*\]", "", line)
    # remove all punctuation
    line = re.sub("[^\w]", " ",  line)
    # strip whitespaces
    line = line.strip()
    return line

### Preprocessing the subtitles files

The subtitles files have a very consistent format and so they are easier to parse. There are some tricky edge cases here (sometimes two characters speak in the same timeframe, sometimes one characters speaks two lines in thes same timeframe) but overall it is easier to parse than the script file. We end up with a dictionary where the keys are specific episodes and the values are a dictionary where the keys are line numbers and the values are the info from that line (start time, end time, character, line). 

In [ ]:
subsfolder = '/Users/Jack/Developer/friends/subs/'
masterpreSQLsubdict = {}

for filename in os.listdir(subsfolder):
    if not filename.startswith('.'):
        # getting season, episode part of filename
        episodesubdict = {}
        info = filename.split('.')[1]
        season = info[1:3]
        episode = info[4:6]
        subspath = subsfolder + filename
        subfile = open(subspath, "r")
        lines = subfile.readlines()
        # using groupby to group raw lines in subtitles files (splitting on lines equal to \n)
        # makes a list with the linenum in first pos, start/end time in second pos, words in 3rd (and 4th) pos
        from itertools import groupby
        linegroups = [list(group) for k, group in groupby(lines, lambda x: x == "\n") if not k]

        for group in linegroups:
            # using regex to get linenum from first line
            linenum = re.findall('\d+', group[0])[0]
            # retrieving start and end times for line
            timesplit = group[1].split()
            starttime = timesplit[0]
            endtime = timesplit[2]

            lineAchar = ""
            lineBchar = ""
            lineA = group[2].strip()
            if len(group) == 3:
                lineB = ""
                lineAchar, lineA = getcharfromline(lineA)
            elif len(group) > 3:
                lineB = group[3].strip()
                # find would return -1 if : is not found
                lineAchar, lineA = getcharfromline(lineA)
                lineBchar, lineB = getcharfromline(lineB)
                lineAstartdash = startswithdash(lineA)
                lineBstartdash = startswithdash(lineB)
                # this is for case where line B is spoken by same person as line A
                if not lineBchar and not lineBstartdash and not lineAstartdash:
                    lineA = lineA + " " + lineB
                    lineB = ""
            else:
                continue
            lineA = preprocess(lineA)
            lineB = preprocess(lineB)
            if lineA and not lineB:
                episodesubdict[linenum] = [season, episode, starttime, endtime, lineAchar, lineA]
            elif lineA and lineB:
                episodesubdict[linenum + "A"] = [season, episode, starttime, endtime, lineAchar, lineA]
                episodesubdict[linenum + "B"] = [season, episode, starttime, endtime, lineBchar, lineB]
            else:
                continue
        masterpreSQLsubdict[season + episode] = episodesubdict

### Inserting organized subtitles data into SQLite database

In [ ]:
# preSQLsublinedict convention is that key is the line number (no a or b or anything), then values is list starting with startime, endtime, x, y, partA line, partB line
# need to update below to account for x and y being added in

# Putting each linepart of subs into subs table in friendsdb database
for seasep, episodeinfo in masterpreSQLsubdict.items():
#     print(episodeinfo)
    for linenum, lineinfo in episodeinfo.items():
        season = lineinfo[0]
        episode = lineinfo[1]
        starttime = lineinfo[2]
        endtime = lineinfo[3]
        char = lineinfo[4]
        line = lineinfo[5]
        x = ""
#         print(char)
        cur.execute('''INSERT OR REPLACE INTO subs (season, episode, starttime, endtime, x, y, linenum, line)
            VALUES ( ?, ?, ?, ?, ?, ?, ?, ? )''', ( season, episode, starttime, endtime, x, char, linenum, line ) )

        conn.commit()